In [2]:
import os 
import sys
import json 
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt 

#ROOT DIRECTORY OF THE PROJECT
ROOT_DIR = os.path.abspath("../../")

#IMPORT OF MASK RCNN
sys.path.append(ROOT_DIR)
from mrcnn.config import Config
from mrcnn import model as modellib,utils

#PATH TO TRAINED WEIGHTS FILE
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR,"mask_rcnn_coco.h5")

#Configurations#

class CustomConfig(Config):
    #Give the configuration a recognizable name
    NAME = "damage"
    # We use a GPU with 12GB memory,which can fit two images
    #Adjust down if you use a smaller GPU.
    IMAGE_PER_GPU = 2
    
    # No. of classes(including background)
    NUM_CLASSES = 1 + 1 #background plus toy
    
    #Number of training steps per epoch
    STEPS_PER_EPOCH = 100
    
    #Skip detections with 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    #######################################DATASET#######################################
    class CustomDataset(utils.Dataset)
    def load_ custom(self, dataset_dir, subset ):
        #Add classes. We have only one class to add.
        assert subset in ["train","val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        
        # load annotation
        # VGG image annotator saves each image in the form:
      #  {'filename': '28503151_5b5b7ec140_b.jpg' ,
       #    'religion': {
        #       '0':{
         #          'religion_attributes': {},
          #         'shape_attributes': {
           #            'all_points_x': [...],
            #           'all_points_y': [...],
             #          'name': 'polygon'}},
              # ...more regions...
     # },
     #'size': 100202 
#}
 # We mostly care about the x and y coordinates of each region
    annotations1 = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
   # print(annotations1)
   annotations = list(annotations1.valves())
    
 #the via tool saves images in the json even if they dont have any
 #annotations.skip unannotated images
    annnotations = [a for a in annotations if a['regions']]
    
 #Add images
 for a in annotations:

        polygons = [r['shape_attributes'] for r in a ['regions'].valves()]
        
    image_path = os.path.join(dataset_dis, a['filename'])
    image = skimage.io.imread(image_path)
    height, width = image.shape[:2]
    
    self.add_image(
        "damage", ##for a single class just add the name here
        image_id = a['filename'], #use file name as a unique image id
        path = image_path,
        width =width, height=height,
         polygons=polygons)
 def load_mask(self, image_id):
    """Generate instance masks for an image.
    Returns:
    masks: A bool array of shape[height,width,instance count] with one mask per instance.
    class_ids: a 1D array of class IDs of the instance masks.
    """
    #If not a balloon dataset image,delegate to parent class.
    image_info = self.image_info[image_id]
    if image_info["source"] != "damage":
        return super(self._class_, self).load_mask(image_id)
    
    #Convert polygons to a bitmap mask of shape
    #[height,width, instance_count]
    info = self.image_info[image_id]
    mask = np.zeros([info["height"], info["width"], len(info["polygons"])], dtype=np.uint8)
    for i,p in enumerate(info["polygons"]):
        #get indexes of pixels inside the polygon and set them to 1
        rr, cc = skimage.draw.polygon(p['all_points_y'],p['all_points_x'])mask[rr,cc,i] = 1
        #return mask, and array of class IDs of eeach instance.Since we have one class Id, we returnan array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)
    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "damage":
            return info["path"]
        else:
            super(self._class_,self).image_reference(image_id)
            
        def train(model):
            """Train the model."""
            #Training dataset.
            dataset_train = CustomDataset()
            dataset_train.load_custom(args.dataset,"train")
            dataset_train.prepare()
            
            #Validation daatset
            dataset_val = CustomDataset()
            dataset_val.load_custom(args.dataset,"val")
            dataset_val.prepare()
            #***This training schedule is an example.
            print("Training network heads")
            model.train(dataset_tarin,dataset_val,
                       learning_rate=config.LEARNING_RATE,
                       epochs=10,
                       layers='heads')
            
            def color_splash(image,mask):
                """Apply color splash effect.
                image: RGB image [height,width,3]
                mask: instance segmetation mask[height,width,instance count]
                Returns result image.
                """
                # Make a grayscale copy of the image. The grayscale copy still 
                #has 3 RGB channels, though.
                gray = skimage.color.gray2rgb(skimage.color.rgb2gray(image))*255
                # We're traeting all instances as one,so collapse the mask into one layer
                mask = (np.sum(mask,-1,keepdims=True) >= 1)
                #Copy color pixels from the original color image where mask is set
                if mask.shape[0] > 0:
                    splash = np.where(mask, image, gray).astype(np.uint8)
                else:
                    splash = gray
                    return splash
                
                def detect_and_color_splash(model, image_path=None, video_path=None):
                    assert image_path or video_path
                    # Image or video?
                    if image_path:
                        #Run model detection and generate the color splash effect
                        print("Running on {}".format(args.image))
                        #Read image
                        image = skimage.io.imread(args.image)
                        #Detect objects
                        r = model.detect([image], verbose=1)[0]
                        # Color splash
                        splash = color_splash(image, r['masks'])
                        # Save output
                        file_name = "splash_{:%Y%m%dT%H%M%S}.png".format(datetime.now())
                        skimage.io.imsave(file_name, splash)
                        elif video_path:
                            import cv2
                            # Video capture
                            vcapture = cv2.VideoCapture(video_path)
                            width = int(vcapture.fet(cv2.CAP_PROP_FRAME_WIDTH))
                            height = int(vcapture.get(cv2.CAP_PROP_FRAME_HEIGHT))
                            fps = vcapture.get(cv2.CAP_PROP_FPF)
                        # Define codec and create video writer
                        file_name ="splash_{:%Y%m%dT%H%M%S}.avi".format(datetime.datetime.now())
                        vwriter - cv2.videoWriter(file_name,
                                                 cv2>VideoWriter_fourcc(*'MJPG')
                                                 fps,(width,height))
                        count = 0
                        success = True
                        while success:
                            print("frame:", count)
                            #Read next image 
                            success, iamge = vcapture.read()
                            if success:
                                # OpenCV returns image as BGR, convert to RGB
                                image = image[...,::-1]
                                # Detect Objects
                                r = model.detect([image],verbose=0)[0]
                                # Color splash 
                                splash = color_splash(image,r['masks'])
                                # RGB -> BGR to save image to video
                                splash = splash[...,::-1]
                                #Add image to video writer
                                vwriter.write(splash)
                                count += 1
                                vwriter.release()
                                print("Saved to",file_name)
                                #########################################TRAINING####################################################
                                
                                if_name_ == '_main_':
                                    import argparse
                                    
                                    #Parse command line arguments
                                    parser = argparse.ArgumentParser(
                                        description = 'Train Mask R-CNN to detect custom class.')
                                    parser.add_argument("command",
                                                       metavar="<command>",
                                                        help="'train' or 'splash'")
                                    parser.add_argument('--dataset',required=False,
                                                       metavar="/path/to/custom/dataset/",
                                                       help='Directory of the custom dataset')
                                    parser.add_argument('--weights', required=True,
                                                       metavar="/path/to/weights.h5",
                                                       help="Path to weights .h5 file or 'coco'")
                                    parser.add_argument('--logs', required=False,
                                                       default=DEFAULT_LOGS_DIR,
                                                       metavar="/path/to/logs",
                                                       help='Logs and checkpoints directory(default=logs/)')
                                    parser.add_argument('--image', required=False,
                                                       metavar="path or URL to image",
                                                       help='Image to apply the color splash effect on')
                                    parser.add_argument('--video', required=False,
                                                       metavar="path or URL to image",
                                                       help='Image to apply the color splash effect on')
                                    args = parser.parse_args()
                                    
                                    #Validate arguments
                                    if args.command == "train":
                                        assert args.dataset,"Argument --dataset is required for training"
                                    elif args.command =="splash":
                                        assert args.image or args.video,\
                                        "Provide --image or --video to apply color splash"
                                        
                                        print("Weights:",args.weights)
                                        print("Dataset:",args.dataset)
                                        print("Logs:",args.logs)
                                        
                                        #Configurations
                                        if args.command =="train":
                                            config = CustomConfig()
                                            else:
                                                class InferenceConfig(CustomConfig):
                                                    # Set batch size to 1 since we'll be running inference on
                                                    # one image at atime.Batch size = GPU_COUNT * IMAGES_PER_GPU
                                                    GPU_COUNT = 1
                                                    IMAGES_PER_GPU = 1
                                                    config = InferenceConfig()
                                                    config.display()
                                                    
                                                    #create model
                                                    if args.command == "train":
                                                        model - modellib.MaskRCNN(mode= "training", config=config,
                                                                                 model_dir=args.logs)
                                                        else:
                                                            model = modellib.MaskRCNN(mode="inference", config=config,
                                                                                     model_dir=args.logs)
                                            # Select weights file to load
                                            if args.weights.lower()== "coco":
                                                weights_path = COCO_WEIGHTS_PATH
                                            
                                            # Dowload weights file
                                            if not os.path.exits(weights_path):
                                                utils.dowload_trained_weights(weights_path)
                                            elif args.weights.lower() == "last":
                                                # Find last trained weights
                                                weights_path = model.find_last()[1]
                                            elif args.weights.lower() == "imagenet":
                                                # Start from Imagenet trained weights
                                                weights_path = model.get_imagenet_weights()
                                            else:
                                                weights_path = args.weights
                                                
                                            # Load weights
                                            print("Loading weights", weights_path)
                                            if args.weights.lower() =="coco"
                                                # Exclude the last layers because they require a matching 
                                                # number of classes
                                                model.load_weights(weights_path,by_name= True, exclude=["mrcnn_class_logits","mrcnn_bbox_fc","mrcnn_box", "mrcnn_mask"])
                                           else:
                                            model.load_weights(weights_path, by_name= True)
                                            
                                            #Train or evaluate
                                            if args.command =="train":
                                                train(model)
                                            elif args.command =="splash":
                                                 detect_and_color_splash(model,image_path=args.image,video_path=args.video)
                                            else:
                                                print("'{}'is not recognized."
                                                     "Use 'train' or 'splash'".format(args.command))
                                    
            
            
        
    ############################SETUP.PY##############################################
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 70)

In [9]:
#####################SETUP.CFG#############################################################3
 
    [metadata]
    description.file = README.md
    license.file = LICENSE
    requirements-file = requirements.txt
    

IndentationError: unexpected indent (<ipython-input-9-ed9155d8a14a>, line 3)

In [10]:
###################################REQUIREMENTS.TXT#########################################
numpy
scipy
pillow
cython
matplotlib
scikit-image
tensorflow>=1.3.0
keras>=2.0.8
opencv-python
h5py
imguag
Ipython[all]

SyntaxError: invalid syntax (<ipython-input-10-277bd1f3ed7c>, line 8)

###################### MANIFEST###########################
include README.md
include LICENSE
include requirements.txt


In [11]:
########################### VISUALIZE.PY#########################################

import os
import sys
import random
import colorsys

import numpy as np
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib.pyplot as plt
from matplotlib import patches, lines
from matplotlib.patches import Polygon
import IPython.display

# ROOT DIRECTORY OF THE PROJECT
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)          # TO FIND LOCAL VERSION OF LIBRARY
from mrcnn import utils



###################################################################### VISUALIZATION #######################################################################
def display_images(images,title=None, cols=4, cmap=None, norm=None, interpolation=None):
    """Display the given set of images, optionally with titles.
    images: list or array of images, optionally with titles.
    titles:optional.A list of titles to display with each image.
    cols: number of images per row
    cmap:Optional. Color map to use. For example, "Blues".
    norm:Optional. A Normalize instance to map valves to colors.
    interpolation/;Optional. Image interpolation to use for display.
    """
    titles - titles if titles is not None else[""]*len(images)
    rows = len(images)// cols + 1
    plt.figure(figsize=(14, 14*rows // cols))
    i = 1
    for image, title in zip(images, titles):
        plt.subplot(rows,cols, i)
        plt.title(title,fontsize=9)
        plt.axis('off')
        plt.imshow(image.astype(np.uint8), cmap=cmap,
                  norm=norm, interpolation=interpolation)
        i += 1
        plt.show()
        
        
        
    def random_colors(N, bright=True):
        """
        Generate random colors.
        To get visually distinct colors, generate them in HSV space then convert to RGB.
        """
        brightness = 1.0 if bright else 0.7
        hsv = [(i /N, 1, brightness) for i in range(N)]
        colors = list(map(Lamda c: colorsys.hsv_to_rgb(*c),hsv))
        random.shuffle(colors)
        return colors
    
    def apply_mask(image, mask, color, alpha=0.5):
        """Apply the given mask to the image.
        """
        for c in range(3):
            image[:, :, c] = np.where(mask ==1,
                                     image[:, :, c]*
                                     (1- alpha) + alpha*color[c]*255,
                                     image[:, :, c])
            return image
        
    def display_instances(images, boxes, masks, class_ids, class_names,
                         scores = None, title = "",
                         figsize=(16, 16), ax=None,
                         show_mask=True, show_bbox=True,
                         colors=None, captions=None):
        """
        boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
        masks: [heigh, width, num_instances]
        class_ids: [num_instances]
        class_names: list of class names of the dataset
        scores: (optional) confidence scores for each box
        title: (optional) Figure title
        show_mask, show_bbox: To show masks and bounding boxes or not
        figsize: (optional) the size of the image
        colors:(optional) An array or colors to use with each objects
        captions: (optional) A list of strings to use as captions for each object
        """
        #Number of instances
        N = boxes.shape[0]
        if not N:
            print("\n*** No instances to display ***\n")
        else:
            assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]
            
        #If no axis is passed, create one and automatically call show()
        auto_show = False
        if not ax:
            _, ax = plt.subplots(1, figsize=figsize)
            auto_show = True
            
        # generate random colors
        
            
        
    

SyntaxError: invalid syntax (<ipython-input-11-54f981822b34>, line 11)